In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import joblib
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# 1. Load Data
df = pd.read_csv("/content/All_dataset_duplicate_dataset.csv")  # Pastikan file memiliki kolom 'full_text'

# 2. Pembersihan Data
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words("indonesian"))

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Hilangkan URL
    text = re.sub(r"[^A-Za-z\s]", "", text)  # Hapus karakter non-alfabet
    text = text.lower()  # Konversi ke huruf kecil
    text = word_tokenize(text)  # Tokenisasi
    text = [word for word in text if word not in stop_words]  # Hilangkan stopwords
    text = [stemmer.stem(word) for word in text]  # Stemming
    # return " ".join(text)
    return text

df["cleaned_text"] = df["full_text"].apply(clean_text)

In [ ]:
# 3. Train Word2Vec Model
model = Word2Vec(sentences=df['cleaned_text'], vector_size=100, window=5, min_count=1, workers=4)
word_vectors = model.wv

# 4. Convert Sentences to Vectors (average word embeddings)
def sentence_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:p
        return np.zeros(model.vector_size)

df['vector'] = df['cleaned_text'].apply(lambda x: sentence_vector(x, model))


X = np.array(df['vector'].tolist())p
X_scaled = StandardScaler().fit_transform(X)  # Normalize data

# K-Means Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

# Manual Labeling of Clusters
cluster_labels = {
    0: 'Positif',
    1: 'Negatif',
    2: 'Netral',
}
df['label'] = df['cluster'].apply(lambda x: cluster_labels.get(x, 'Neutral'))


In [ ]:
df['label'].value_counts()

,count
label,
Positif,2615
Negatif,2571
Netral,454


In [ ]:
df[['cleaned_text', 'label']]

,cleaned_text,label
0,sisi gelap menang farhan alkatiri dprd muda ra...,Negatif
1,menolaklupa tiputipu ridwan kamil rk undang wa...,Positif
2,bismillah ridwan kamil suswono jakarta maju,Negatif
3,ridwan kamil pramono anung anies baswedan nget...,Negatif
4,rabu agustus pasang rido ridwan kamil suswono ...,Positif
...,...,...
5635,sendyyeah sigantengkalem wakil bssn komjen dha...,Negatif
5636,serikat buruh excopartaiburuh daerah khusus ja...,Negatif
5637,senator jakarta fahiraidris dukung rido paslon...,Negatif
5638,mantap dpd pks jaktim hadir serta acara konsol...,Negatif


In [ ]:
# 4. Persiapan Data untuk Training
X = df["cleaned_text"]
y = df["label"]

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Vektorisasi Teks
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# 6. Training Model Naive Bayes
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# 7. Evaluasi Model
y_pred = model.predict(X_test_vectorized)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 8. Simpan Model dan Vectorizer
joblib.dump(model, "naivebayes_sentiment_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

print("Model dan vectorizer berhasil disimpan!")

Accuracy: 0.7606382978723404

Classification Report:
               precision    recall  f1-score   support

     Negatif       0.76      0.77      0.77       512
      Netral       0.93      0.58      0.72        96
     Positif       0.74      0.78      0.76       520

    accuracy                           0.76      1128
   macro avg       0.81      0.71      0.75      1128
weighted avg       0.77      0.76      0.76      1128

Model dan vectorizer berhasil disimpan!


In [ ]:
# 1. Load Data
df1 = pd.read_csv("/content/All_dataset_duplicate_dataset.csv")  # Pastikan file memiliki kolom 'full_text'

In [ ]:
df1["cleaned_text"] = df["cleaned_text"].apply(lambda x: " ".join(x))
df1.to_csv("Clean_All_dataset_duplicate_dataset.csv", index=False)

In [ ]:
def token_text(text):
    text = word_tokenize(text)  # Tokenisasi
    return text

In [ ]:
# df['cleaned_text'] = df['cleaned_text'].apply(token_text)